### Knet.jl Implementation of Deep Reparametrization 

This notebook contains an attempt at implementing deep reparametrization of curves using Knet.jl. It is currently not working as hoped, due to problems with using the Autograd package.

The implementation using Flux.jl (in another notebook) saw more success. 

In [1]:
using Knet
using Plots
using Reparam
using LinearAlgebra

In [2]:
module dev
using Knet

function squaresum(x)
    out = x[1]^2
    for i in 2:length(x)
        out += x[i]^2
    end
    out
end

function vecnorm2(x)
    return sqrt(squaresum(x))
end


struct Qmap{T<:Function, TF<:Real}; c::T; h::TF; end
(q::Qmap)(x) = sqrt(vecnorm2(0.5 * (q.c(x+q.h) - q.c(x-q.h)) / q.h)) * q.c(x)


struct SineLayer; N; c; end 
SineLayer(c) = SineLayer(length(c), Param(c))

function (S::SineLayer)(x, y)
    z = sin.(π*x) * S.c[1]
    y1 = π * cos.(π*x) * S.c[1]
    for n in 2:S.N
        z += sin.(n*π*x) * S.c[n]
        y1 += n * π * cos.(n*π*x) * S.c[n]
    end
    return x .+ z, (1. .+ y1 ) .* y
end

(S::SineLayer)(x::AbstractVector) = S(x, ones(size(x)...))
(S::SineLayer)(x) = S(x...)

struct Chain; layers; Chain(args...) = new(args); end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
function (c::Chain)(x, r)
    y=x
    for l in c.layers
        y = l(y)
    end
    z, y = y
    println.((z, y))
    sqrt.(y) .* r.(z)
end

function (c::Chain)(x, q, r)
    z, y = c(x)
    Q = q.(x)
    println(size(Q))
    println(size(z))
    println(size(y))
    println(size(sqrt.(y)))
    println(size(r.(z)))
    R = sqrt.(y) .* r.(z)
    println.((size(Q), size(R)))
    E = q.(x) - sqrt.(y) .* r.(z)
    sum(squaresum.(E))
end
end # module

Main.dev

In [3]:
# Define test curves and reparametrization
c(t) = [cos(2π*t), sin(2π*t)]
γ(t) = 0.9t^2 + 0.1t
ψ(t) = 0.5 * log(20t + 1) / log(21) + 0.25 * (1 + tanh(20*(t-0.5))/tanh(21))
# c2 = c ∘ γ
c2 = c∘ψ

# Get Q-maps
q = dev.Qmap(c, 1e-4)
r = dev.Qmap(c2, 1e-4)


# Define a "batch" of K test points
K = 4
X = range(0, 1, length=K)


# Define a SineLayer with N basis functions
N = 2
S = dev.SineLayer(zeros(N))


# Define a chain model with single sinelayer
model = dev.Chain(dev.SineLayer(0.1*ones(N)))

Main.dev.Chain((Main.dev.SineLayer(2, P(Array{Float64,1}(2))),))

In [4]:
Q = q.(X)

4-element Array{Array{Float64,1},1}:
 [2.506628192166236, 0.0]
 [-1.2533140960830362, 2.170803692258082]
 [-1.2533140960832372, -2.170803692258426]
 [2.506628192166512, -6.139468384378472e-16]

In [5]:
Z, Y = model(X)

([0.0, 0.506538414090221, 0.6666666666666666, 1.0], [1.942477796076938, 0.8429203673205106, 0.5287611019615308, 1.3141592653589793])

In [6]:
r.(Z)

4-element Array{Array{Float64,1},1}:
 [4.542871816332296, 2.9416481144703133e-8]
 [-2.4929078683336274, -5.150164275760763]
 [1.1657567171920322, -0.4902722041884899]
 [0.9913357194507286, -6.419202003260748e-9]

In [7]:
r.(X)

4-element Array{Array{Float64,1},1}:
 [4.542871816332296, 2.9416481144703133e-8]
 [-0.8609518048126881, 1.4526356044536957]
 [1.1657567171920322, -0.4902722041884899]
 [0.9913357194507286, -6.419202003260748e-9]

In [8]:
R = sqrt.(Y) .* r.(Z)

4-element Array{Array{Float64,1},1}:
 [6.331527745510464, 4.0998574045909125e-8]
 [-2.2887560337648036, -4.728401603107244]
 [0.847691202670793, -0.3565061459870112]
 [1.136435405050931, -7.358766849157151e-9]